### 🍽️ Scraping inicial de TripAdvisor
Este bloque descarga todos los restaurantes de una ciudad (ejemplo: **A Coruña**) usando la API de TripAdvisor vía RapidAPI.  
- Primero obtiene el listado paginado de restaurantes.  
- Después, para cada restaurante, intenta recuperar los **detalles completos** (dirección, rating, reseñas, coordenadas, etc.).  
- Si falla el detalle, se guarda una versión mínima con los datos básicos.  
- Finalmente, se genera un archivo `restaurantes_<ciudad>_completo.json` con todos los registros encontrados.  


In [ ]:
import requests
import json
import time

# PARÁMETRO: cambiar esto para consultar otra ciudad o provincia
CITY_QUERY = "a coruña"

# Configuración general
LIST_URL = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/list"
DETAIL_URL = "https://tripadvisor-scraper.p.rapidapi.com/restaurants/detail"

HEADERS = {
    "x-rapidapi-key": "",
    "x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

all_details = []
page = 1

print(f"🔍 Iniciando búsqueda de restaurantes en: {CITY_QUERY}")

# Paso 1: Obtener la primera página para saber cuántas hay
params = {"query": CITY_QUERY, "page": str(page)}
response = requests.get(LIST_URL, headers=HEADERS, params=params)

if response.status_code != 200:
    print(f"❌ Error inicial en la solicitud: {response.status_code}")
    print(response.text)
    exit()

data = response.json()
total_pages = data.get("total_pages", 1)
print(f"📄 Total de páginas detectadas: {total_pages}")

# Paso 2: Recorrer todas las páginas del listado
for page in range(1, total_pages + 1):
    print(f"\n📦 Procesando página {page}/{total_pages}...")
    params["page"] = str(page)
    response = requests.get(LIST_URL, headers=HEADERS, params=params)

    if response.status_code != 200:
        print(f"⚠️ Error en la página {page}: {response.status_code}")
        continue

    list_data = response.json()
    results = list_data.get("results", [])

    for rest in results:
        rest_id = rest.get("id")
        if not rest_id:
            continue

        detail_params = {"id": str(rest_id)}
        detail_response = requests.get(DETAIL_URL, headers=HEADERS, params=detail_params)

        if detail_response.status_code == 200:
            detailed_data = detail_response.json()
            all_details.append(detailed_data)
            print(f"✅ Restaurante ID {rest_id} procesado.")
        else:
            print(f"❌ Error al obtener detalle del restaurante ID {rest_id}: {detail_response.status_code}")
            # Guarda datos mínimos del listado si falla el detalle
            fallback_data = {
                "id": rest.get("id"),
                "name": rest.get("name"),
                "link": rest.get("link"),
                "rating": rest.get("rating"),
                "reviews": rest.get("reviews"),
                "price_range_usd": rest.get("price_range_usd"),
                "latitude": rest.get("latitude"),
                "longitude": rest.get("longitude"),
                "cuisines": rest.get("cuisines"),
                "is_sponsored": rest.get("is_sponsored"),
                "is_premium": rest.get("is_premium"),
                "menu_link": rest.get("menu_link"),
                "reservation_link": rest.get("reservation_link"),
                "featured_image": rest.get("featured_image"),
                "detalle_completo": False  # Marca que es un dato incompleto
            }
            all_details.append(fallback_data)

        time.sleep(0.5)  # Espera para respetar la cuota de la API

# Paso 4: Guardar en un archivo JSON personalizado
safe_city = CITY_QUERY.lower().replace(" ", "_")
filename = f"restaurantes_{safe_city}_completo.json"

with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_details, f, ensure_ascii=False, indent=4)

print(f"\n🎉 Archivo guardado como '{filename}' con {len(all_details)} restaurantes.")


🔍 Iniciando búsqueda de restaurantes en: a coruña
📄 Total de páginas detectadas: 40

📦 Procesando página 1/40...
✅ Restaurante ID 3326098 procesado.
✅ Restaurante ID 6642068 procesado.
✅ Restaurante ID 10337492 procesado.
✅ Restaurante ID 3164949 procesado.
✅ Restaurante ID 20928703 procesado.
✅ Restaurante ID 850438 procesado.
✅ Restaurante ID 2024162 procesado.
✅ Restaurante ID 2187063 procesado.
✅ Restaurante ID 12535025 procesado.
✅ Restaurante ID 19516381 procesado.
✅ Restaurante ID 6622736 procesado.
✅ Restaurante ID 5122422 procesado.
✅ Restaurante ID 14929254 procesado.
✅ Restaurante ID 992647 procesado.
✅ Restaurante ID 992683 procesado.
✅ Restaurante ID 14177323 procesado.
✅ Restaurante ID 992610 procesado.
✅ Restaurante ID 1550209 procesado.
✅ Restaurante ID 14004784 procesado.
✅ Restaurante ID 16609217 procesado.
✅ Restaurante ID 8773188 procesado.
✅ Restaurante ID 11897721 procesado.
✅ Restaurante ID 992612 procesado.
✅ Restaurante ID 6782476 procesado.
✅ Restaurante ID 86